<a href="https://colab.research.google.com/github/ioradlin/NAT_project/blob/Encoder/NAT_Encoder_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def to_txt(x, name):
  with open(name, 'w') as file:
      file.write(x)

def read_txt(name):
  with open(name, 'r') as file:
      temp= file.read()
  return temp

## Transformer model

based on : https://towardsdatascience.com/build-your-own-transformer-from-scratch-using-pytorch-84c850470dcb

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

In [3]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output

    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))

        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [4]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [5]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

#### Encoder

In [6]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

#### Decoder

In [7]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [8]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):

      # src mask
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
      # tgt mask
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        #output= nn.functional.softmax(output, dim=2)

        return output

In [9]:
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (64, 30))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, 40))  # (batch_size, seq_length)

In [10]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(5):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

## NAT ENcoder

In [11]:
import torch
import torch.nn as nn

class NATEncoder(nn.Module):
    def __init__(self, src_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(NATEncoder, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fertility_fc = nn.Linear(d_model, 50) #L=50 from NAT paper
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src):

        # src mask
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        return src_mask

    def forward(self, src):
        src_mask = self.generate_mask(src)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        output = self.fertility_fc(enc_output)
        return output

In [12]:
#Encoder (fertility predictor)
src_vocab_size = 1000; d_model = 512; num_heads = 8; num_layers = 6
d_ff = 2048; max_seq_length = 100; dropout = 0.1

encoder = NATEncoder(src_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

## Encoder용 data preprocess

#### Load datasets

In [13]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00


In [ ]:
# from datasets import load_dataset

# dataset = load_dataset("bbaaaa/iwslt14-de-en-preprocess")

#### Load corpus, vocab, fertility label

In [14]:
import pickle

with open('train_ft_bpe.pkl', 'rb') as file:
     ft = pickle.load(file)

with open('src_vocab.pkl', 'rb') as file:
     src_vocab = pickle.load(file)

with open('tgt_vocab.pkl','rb') as file:
     tgt_vocab = pickle.load(file)

In [18]:
## BPE tokenized

bpe_de= read_txt('bpe_de.txt')
bpe_en= read_txt('bpe_en.txt')

#### **dataset**

In [19]:
de= bpe_de.split('\n')
en= bpe_en.split('\n')

In [20]:
from collections import Counter
lengths= [len(d) for d in de]

i = 0
for length in lengths:
  if length<=512:
    i+=1

i/len(lengths)

## 99% 이상이 512 이상 길이이므로, 512를 max_len으로 사용

0.9959310779523087

#### Preprocess

id로 전환, padding & truncation으로 max_len 통일

In [21]:
def tokens_to_ids(text, vocab, max_seq_len=512):
    preprocessed = []

    for line in text:
        tokens = line.split()
        ids = [vocab.get(token, vocab.get('[UNK]')) for token in tokens]

        # padding & trunc는 collate_fn에서 처리! (batch별로 local max_len 써야 효율적)

        # if len(ids) > max_seq_len:
        #     ids = ids[:max_seq_len]
        # else:
        #     ids.extend([vocab['[PAD]']] * (max_seq_len - len(ids)))

        preprocessed.append(ids)

    return preprocessed

In [22]:
tokened_de= tokens_to_ids(de, src_vocab)
tokened_en= tokens_to_ids(en, tgt_vocab)

In [27]:
data=[]
for i in range(len(ft)):
  data.append({'de':tokened_de[i],'en':tokened_en[i],'ft':ft[i]})

#### ✰✰ Loader & collate_fn ✰✰

In [41]:
from torch.nn.utils.rnn import pad_sequence

def tensor_pad_collate_fn(batch):
    padded_data = {}

    # Calculate max_len for each key separately
    max_lens = {key: min(max(len(item[key]) for item in batch if key in item), 512) for key in batch[0].keys()}

    for key in batch[0].keys():
        if key in max_lens:
            max_len = max_lens[key]
            sequences = [torch.tensor(item[key][:max_len] if len(item[key]) > max_len else item[key] + [0] * (max_len - len(item[key])), dtype=torch.long) for item in batch if key in item]
            padded_sequences = torch.nn.utils.rnn.pad_sequence(sequences, batch_first=True, padding_value=0)
            padded_data[key] = padded_sequences[:, :max_len]  # Truncate to max_len if longer

    return padded_data

In [53]:
from torch.utils.data import DataLoader

batch_size = 32
train_loader = DataLoader(data, batch_size=batch_size, shuffle=True, collate_fn= tensor_pad_collate_fn)

In [52]:
## shape check ! (de와 ft는 같아야함)

for _ in train_loader:
  for key in _.keys():
    print({key}, _[key].shape)
  break

{'de'} torch.Size([32, 48])
{'en'} torch.Size([32, 47])
{'ft'} torch.Size([32, 48])


## Encoder Training (with gpu)

In [57]:
#TransformerEncoder init & to cuda

src_vocab_size = len(src_vocab); d_model = 512; num_heads = 8; num_layers = 6
d_ff = 2048; max_seq_length = 512; dropout = 0.1

encoder = NATEncoder(src_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)

print(device)

cuda


In [63]:
criterion = nn.CrossEntropyLoss(ignore_index=0) #0: special token / softmax 계산을 내장하므로 unnormalized logits(scroes) 그대로 씀..!!
optimizer = optim.Adam(encoder.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9)
train_loader = DataLoader(data, batch_size=batch_size, shuffle=True, collate_fn= tensor_pad_collate_fn)

encoder.train()

for epoch in range(3):

    for i, batch in enumerate(train_loader):

        optimizer.zero_grad()

        src = batch['de'].to(device)
        ft = batch['ft'].to(device)

        if src.shape != ft.shape:
          print('hi')
          print(i, src.shape, ft.shape)
          continue

        output = encoder(src)

        # L= 50 (fertility 범위: 0~49) from paper
        loss = criterion(output.contiguous().view(-1,50), ft.contiguous().view(-1))
        loss.backward()
        optimizer.step()


    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

hi
0 torch.Size([32, 71]) torch.Size([32, 70])
hi
4 torch.Size([32, 62]) torch.Size([32, 60])
hi
53 torch.Size([32, 49]) torch.Size([32, 48])
hi
70 torch.Size([32, 90]) torch.Size([32, 89])
hi
128 torch.Size([32, 52]) torch.Size([32, 51])
hi
148 torch.Size([32, 67]) torch.Size([32, 66])
hi
204 torch.Size([32, 72]) torch.Size([32, 71])
hi
247 torch.Size([32, 74]) torch.Size([32, 73])
hi
365 torch.Size([32, 65]) torch.Size([32, 63])
hi
416 torch.Size([32, 72]) torch.Size([32, 71])
hi
442 torch.Size([32, 45]) torch.Size([32, 44])
hi
447 torch.Size([32, 62]) torch.Size([32, 60])
hi
456 torch.Size([32, 47]) torch.Size([32, 46])
hi
493 torch.Size([32, 61]) torch.Size([32, 60])
hi
642 torch.Size([32, 119]) torch.Size([32, 118])
hi
754 torch.Size([32, 89]) torch.Size([32, 88])
hi
827 torch.Size([32, 63]) torch.Size([32, 62])
hi
887 torch.Size([32, 95]) torch.Size([32, 94])
hi
962 torch.Size([32, 72]) torch.Size([32, 71])
hi
977 torch.Size([32, 87]) torch.Size([32, 86])
hi
1051 torch.Size([32, 

KeyboardInterrupt: ignored

### Example (transformer)

In [ ]:
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 5
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (16, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (16, max_seq_length))  # (batch_size, seq_length)

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=0) #0: special token / softmax 계산을 내장하므로 unnormalized logits(scroes) 그대로 씀..!!
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(100):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data)

    loss = criterion(output.contiguous().view(-1,tgt_vocab_size), tgt_data.contiguous().view(-1))

    # print(output.contiguous().view(-1,tgt_vocab_size).shape, tgt_data.contiguous().view(-1).shape)
    # print('loss',loss)
    loss.backward()
    optimizer.step()
    # break
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")